In [1]:
# should already be activated, but just a reminder where ya are :)  
# !conda activate wool_sucking_nn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 

%matplotlib inline

In [3]:
# import data
excel_file = pd.read_excel('..\Data\cleaned_and_split_data.xlsx', sheet_name=None)

# Unpack the dataframes into separate variables
X_train = excel_file['X_train']
X_test = excel_file['X_test']
y_train = excel_file['y_train']
y_test = excel_file['y_test']

In [4]:
print(X_train.shape)
X_train.head()

(3965, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,2,4,4,3
1,1,HCS,1,2,4,4,2
2,0,ORI,1,1,1,1,1
3,1,MCO,1,1,2,2,3
4,1,HCS,1,1,2,1,2


In [5]:
print(X_test.shape)
X_test.head()

(1700, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,3,3,1,1
1,1,MCO,1,1,2,1,1
2,1,RUS,1,1,4,4,1
3,0,EUR,2,2,2,2,1
4,1,NFO,1,2,3,2,1


In [6]:
print(y_train.shape)
y_train.head()

(3965, 1)


,Wool_sucking_binary
0,0
1,0
2,0
3,0
4,0


In [7]:
print(y_test.shape)
y_test.head()

(1700, 1)


,Wool_sucking_binary
0,1
1,1
2,1
3,0
4,1


## To record model metrics

In [8]:
# dataframe for scores amongst models
master_scores = pd.DataFrame(columns=['Model','Recall', 'F1', 'Precision', 'Accuracy'])

In [9]:
# add_to_master(model_description, y_true, y_predicted, binary=True)
def add_to_master(model_description, y_test, y_pred, binary=True):
    '''
    Adds a new row to the running score DataFrame `master_scores` 
    and functions for adding both binary and multiclass scores. 
    '''
    global master_scores
    if binary == True:
        # Evalution Metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # add to master_scores
        new_row = {
            'Model': model_description,
            'Recall': round(recall,3), 
            'F1': round(f1, 3), 
            'Precision': round(precision, 3), 
            'Accuracy': round(accuracy, 3)}
        
        master_scores = master_scores.append(new_row, ignore_index=True)   
    
    else:
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro') # I chose macro since we have less of the sucklers 
        recall = recall_score(y_test, y_pred, average='macro')       # and I want their accuracy to be meaningful
        f1 = f1_score(y_test, y_pred, average='macro')


        # add to master_scores
        new_row = {
            'Model': model_description,
            'Recall': round(recall,3), 
            'F1': round(f1, 3), 
            'Precision': round(precision, 3), 
            'Accuracy': round(accuracy, 3)}
        master_scores = master_scores.append(new_row, ignore_index=True)
    return master_scores

# Prep for tensorflow.keras

In [10]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
# for reproducibility 
tf.random.set_seed(42)
np.random.seed(42)

In [11]:
# one-hot breed 
X_train_encoded = pd.get_dummies(X_train, columns=['Breed_group'])
X_test_encoded = pd.get_dummies(X_test, columns=['Breed_group'])

In [12]:
X_train_encoded

,Neuter_status,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming,Breed_group_ABY,Breed_group_BEN,Breed_group_BRI,Breed_group_BUR,...,Breed_group_MCO,Breed_group_NFO,Breed_group_ORI,Breed_group_PER,Breed_group_RAG,Breed_group_RUS,Breed_group_SBI,Breed_group_SIB,Breed_group_TUV,Breed_group_other
0,1,1,2,4,4,3,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,1,2,4,4,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,1,1,2,2,3,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,1,1,2,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3960,1,1,3,1,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3961,1,2,2,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3962,1,1,1,3,4,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3963,1,1,1,4,4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Plain Jane

In [13]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
def recall(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

tf.keras.utils.get_custom_objects()['recall'] = recall

In [15]:
model.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [16]:
model.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 1s 941us/step - loss: 0.6306 - accuracy: 0.6701 - recall: 0.0478
Epoch 2/10
124/124 [==============================] - 0s 892us/step - loss: 0.6047 - accuracy: 0.6888 - recall: 0.1033
Epoch 3/10
124/124 [==============================] - 0s 884us/step - loss: 0.5947 - accuracy: 0.6948 - recall: 0.1883
Epoch 4/10
124/124 [==============================] - 0s 916us/step - loss: 0.5908 - accuracy: 0.6933 - recall: 0.1611
Epoch 5/10
124/124 [==============================] - 0s 884us/step - loss: 0.5862 - accuracy: 0.6943 - recall: 0.1877
Epoch 6/10
124/124 [==============================] - 0s 892us/step - loss: 0.5850 - accuracy: 0.6948 - recall: 0.2115
Epoch 7/10
124/124 [==============================] - 0s 884us/step - loss: 0.5838 - accuracy: 0.6979 - recall: 0.2196
Epoch 8/10
124/124 [==============================] - 0s 876us/step - loss: 0.5791 - accuracy: 0.7032 - recall: 0.2156
Epoch 9/10
124/124 [============================

In [17]:
test_loss, test_acc, test_recall = model.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 715us/step - loss: 0.5896 - accuracy: 0.6812 - recall: 0.2342
Test Accuracy: 0.681176483631134
Test Loss: 0.5896172523498535
Test Recall: 0.2341715693473816


In [18]:
y_pred = model.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
# add_to_master(model_description, y_true, y_predicted, binary=True)
add_to_master("1 hidden, 64 nodes per, adam", y_test, y_pred)
cm

54/54 [==============================] - 0s 621us/step


C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1025,  134],
       [ 408,  133]], dtype=int64)

In [19]:
master_scores

,Model,Recall,F1,Precision,Accuracy
0,"1 hidden, 64 nodes per, adam",0.246,0.329,0.498,0.681


Not doing too hot, but also seems to be doing better than initial models in Cat-pstone 1. 

## Adagrad optimizer

In [20]:
model2 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [21]:
# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model2.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [22]:
model2.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 0s 827us/step - loss: 0.6365 - accuracy: 0.6815 - recall: 0.0041
Epoch 2/10
124/124 [==============================] - 0s 835us/step - loss: 0.6206 - accuracy: 0.6815 - recall: 0.0260
Epoch 3/10
124/124 [==============================] - 0s 835us/step - loss: 0.6129 - accuracy: 0.6918 - recall: 0.0926
Epoch 4/10
124/124 [==============================] - 0s 843us/step - loss: 0.6080 - accuracy: 0.6928 - recall: 0.0864
Epoch 5/10
124/124 [==============================] - 0s 851us/step - loss: 0.6043 - accuracy: 0.6926 - recall: 0.1008
Epoch 6/10
124/124 [==============================] - 0s 860us/step - loss: 0.6009 - accuracy: 0.6938 - recall: 0.1180
Epoch 7/10
124/124 [==============================] - 0s 819us/step - loss: 0.5986 - accuracy: 0.6898 - recall: 0.1175
Epoch 8/10
124/124 [==============================] - 0s 851us/step - loss: 0.5969 - accuracy: 0.6926 - recall: 0.1205
Epoch 9/10
124/124 [============================

In [23]:
test_loss, test_acc, test_recall = model2.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 715us/step - loss: 0.5964 - accuracy: 0.6888 - recall: 0.1738
Test Accuracy: 0.6888235211372375
Test Loss: 0.596362829208374
Test Recall: 0.1737835854291916


In [24]:
y_pred = model2.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adagrad", y_test, y_pred)
cm

54/54 [==============================] - 0s 583us/step


C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1074,   85],
       [ 444,   97]], dtype=int64)

## Scaled with Plain Jane and Adagrad

In [25]:
scaler = StandardScaler()
scaler.fit(X_train_encoded)

StandardScaler()

In [26]:
X_train_scaled_encoded = scaler.transform(X_train_encoded)
X_test_scaled_encoded = scaler.transform(X_test_encoded)

In [27]:
X_train_scaled_encoded.shape

(3965, 25)

In [28]:
# Plain Jane + scaled
model3 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model3.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model3.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model3.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model3.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adam, scaled", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 876us/step - loss: 0.6195 - accuracy: 0.6673 - recall: 0.0564
Epoch 2/10
124/124 [==============================] - 0s 884us/step - loss: 0.5916 - accuracy: 0.6898 - recall: 0.1390
Epoch 3/10
124/124 [==============================] - 0s 884us/step - loss: 0.5840 - accuracy: 0.6989 - recall: 0.2208
Epoch 4/10
124/124 [==============================] - 0s 892us/step - loss: 0.5804 - accuracy: 0.7004 - recall: 0.1852
Epoch 5/10
124/124 [==============================] - 0s 892us/step - loss: 0.5745 - accuracy: 0.7024 - recall: 0.2152
Epoch 6/10
124/124 [==============================] - 0s 908us/step - loss: 0.5735 - accuracy: 0.7044 - recall: 0.2289
Epoch 7/10
124/124 [==============================] - 0s 892us/step - loss: 0.5709 - accuracy: 0.7019 - recall: 0.2356
Epoch 8/10
124/124 [==============================] - 0s 908us/step - loss: 0.5669 - accuracy: 0.7062 - recall: 0.2498
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[998, 161],
       [388, 153]], dtype=int64)

In [29]:
# adagrad + scaled 
model4 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model4.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model4.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model4.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model4.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 64 nodes per, adagrad, scaled", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 803us/step - loss: 0.6154 - accuracy: 0.6729 - recall: 0.0646
Epoch 2/10
124/124 [==============================] - 0s 827us/step - loss: 0.5928 - accuracy: 0.6888 - recall: 0.0752
Epoch 3/10
124/124 [==============================] - 0s 819us/step - loss: 0.5866 - accuracy: 0.6923 - recall: 0.1415
Epoch 4/10
124/124 [==============================] - 0s 819us/step - loss: 0.5834 - accuracy: 0.6921 - recall: 0.1377
Epoch 5/10
124/124 [==============================] - 0s 827us/step - loss: 0.5812 - accuracy: 0.6938 - recall: 0.1502
Epoch 6/10
124/124 [==============================] - 0s 851us/step - loss: 0.5793 - accuracy: 0.6948 - recall: 0.1661
Epoch 7/10
124/124 [==============================] - 0s 827us/step - loss: 0.5778 - accuracy: 0.6963 - recall: 0.1668
Epoch 8/10
124/124 [==============================] - 0s 819us/step - loss: 0.5766 - accuracy: 0.7009 - recall: 0.1844
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1063,   96],
       [ 446,   95]], dtype=int64)

In [30]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
2,"1 hidden, 64 nodes per, adam, scaled",0.283,0.358,0.487,0.677
0,"1 hidden, 64 nodes per, adam",0.246,0.329,0.498,0.681
1,"1 hidden, 64 nodes per, adagrad",0.179,0.268,0.533,0.689
3,"1 hidden, 64 nodes per, adagrad, scaled",0.176,0.260,0.497,0.681


In [31]:
def create_mlp(nodes, optimizer, X_train):
    # Define the model architecture
    model = keras.Sequential([
    keras.layers.Dense(nodes, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(nodes, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

    model.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])
    
    return model

In [32]:
model5 = create_mlp(100, 'adam', X_train_scaled_encoded)

In [33]:
model5.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model5.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model5.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 100 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 981us/step - loss: 0.6053 - accuracy: 0.6817 - recall: 0.1278
Epoch 2/10
124/124 [==============================] - 0s 997us/step - loss: 0.5872 - accuracy: 0.6946 - recall: 0.1965
Epoch 3/10
124/124 [==============================] - 0s 989us/step - loss: 0.5809 - accuracy: 0.6994 - recall: 0.2357
Epoch 4/10
124/124 [==============================] - 0s 989us/step - loss: 0.5777 - accuracy: 0.7009 - recall: 0.2034
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5718 - accuracy: 0.7077 - recall: 0.2370
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5718 - accuracy: 0.7072 - recall: 0.2567
Epoch 7/10
124/124 [==============================] - 0s 989us/step - loss: 0.5702 - accuracy: 0.7042 - recall: 0.2627
Epoch 8/10
124/124 [==============================] - 0s 997us/step - loss: 0.5652 - accuracy: 0.7125 - recall: 0.2643
Epoch 9/10
124/124 [==============================] 

C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[992, 167],
       [375, 166]], dtype=int64)

In [34]:
model6 = create_mlp(32, 'adam', X_train_scaled_encoded)
model6.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model6.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model6.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 32 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 835us/step - loss: 0.6253 - accuracy: 0.6797 - recall: 0.0208
Epoch 2/10
124/124 [==============================] - 0s 879us/step - loss: 0.5958 - accuracy: 0.6868 - recall: 0.0700
Epoch 3/10
124/124 [==============================] - 0s 860us/step - loss: 0.5888 - accuracy: 0.6910 - recall: 0.1674
Epoch 4/10
124/124 [==============================] - 0s 851us/step - loss: 0.5838 - accuracy: 0.6974 - recall: 0.1710
Epoch 5/10
124/124 [==============================] - 0s 868us/step - loss: 0.5805 - accuracy: 0.6963 - recall: 0.1854
Epoch 6/10
124/124 [==============================] - 0s 924us/step - loss: 0.5787 - accuracy: 0.6981 - recall: 0.2013
Epoch 7/10
124/124 [==============================] - 0s 876us/step - loss: 0.5765 - accuracy: 0.6981 - recall: 0.1945
Epoch 8/10
124/124 [==============================] - 0s 884us/step - loss: 0.5738 - accuracy: 0.7032 - recall: 0.2294
Epoch 9/10
124/124 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[1052,  107],
       [ 436,  105]], dtype=int64)

In [35]:
model7 = create_mlp(250, 'adam', X_train_scaled_encoded)
model7.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model7.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model7.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 250 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 1s 1ms/step - loss: 0.6063 - accuracy: 0.6830 - recall: 0.1546
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5873 - accuracy: 0.6951 - recall: 0.2081
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5801 - accuracy: 0.6984 - recall: 0.2321
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5780 - accuracy: 0.7006 - recall: 0.2032
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5696 - accuracy: 0.7082 - recall: 0.2384
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5722 - accuracy: 0.7072 - recall: 0.2569
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5682 - accuracy: 0.7107 - recall: 0.2743
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5618 - accuracy: 0.7140 - recall: 0.2772
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[984, 175],
       [393, 148]], dtype=int64)

In [36]:
# definitely overfits
model8 = create_mlp(300, 'adam', X_train_scaled_encoded)
model8.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model8.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model8.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("1 hidden, 300 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 1s 1ms/step - loss: 0.6081 - accuracy: 0.6837 - recall: 0.1785
Epoch 2/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5880 - accuracy: 0.6946 - recall: 0.2097
Epoch 3/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5795 - accuracy: 0.6989 - recall: 0.2284
Epoch 4/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5771 - accuracy: 0.7026 - recall: 0.2071
Epoch 5/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5687 - accuracy: 0.7107 - recall: 0.2436
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5706 - accuracy: 0.7077 - recall: 0.2559
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5671 - accuracy: 0.7087 - recall: 0.2751
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5613 - accuracy: 0.7117 - recall: 0.2688
Epoch 9/10
124/124 [==============================] - 0s 1ms/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[980, 179],
       [394, 147]], dtype=int64)

In [37]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
4,"1 hidden, 100 nodes, adam",0.307,0.380,0.498,0.681
2,"1 hidden, 64 nodes per, adam, scaled",0.283,0.358,0.487,0.677
6,"1 hidden, 250 nodes, adam",0.274,0.343,0.458,0.666
7,"1 hidden, 300 nodes, adam",0.272,0.339,0.451,0.663
0,"1 hidden, 64 nodes per, adam",0.246,0.329,0.498,0.681
5,"1 hidden, 32 nodes, adam",0.194,0.279,0.495,0.681
1,"1 hidden, 64 nodes per, adagrad",0.179,0.268,0.533,0.689
3,"1 hidden, 64 nodes per, adagrad, scaled",0.176,0.260,0.497,0.681


Okay so we are pretty similar in performance to the logistic regression models from cat-pstone 1 on every metric except recall (and the part it plays in F1, of course). Let's see if over or undersampling with the top performing model architecture (model1). 

In [38]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [39]:
rus = RandomUnderSampler()
X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train_scaled_encoded, y_train)

ros = RandomOverSampler()
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_scaled_encoded, y_train)

In [40]:
# RUS 
model9 = create_mlp(64, 'adam', X_train_scaled_encoded)
model9.fit(X_resampled_rus, y_resampled_rus, epochs=10, batch_size=32)

y_pred = model9.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("RUS 1 hidden, 64 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
79/79 [==============================] - 0s 869us/step - loss: 0.6725 - accuracy: 0.5828 - recall: 0.5245
Epoch 2/10
79/79 [==============================] - 0s 895us/step - loss: 0.6499 - accuracy: 0.6335 - recall: 0.5914
Epoch 3/10
79/79 [==============================] - 0s 908us/step - loss: 0.6410 - accuracy: 0.6371 - recall: 0.6219
Epoch 4/10
79/79 [==============================] - 0s 882us/step - loss: 0.6350 - accuracy: 0.6434 - recall: 0.6260
Epoch 5/10
79/79 [==============================] - 0s 895us/step - loss: 0.6302 - accuracy: 0.6529 - recall: 0.6315
Epoch 6/10
79/79 [==============================] - 0s 895us/step - loss: 0.6263 - accuracy: 0.6529 - recall: 0.6697
Epoch 7/10
79/79 [==============================] - 0s 895us/step - loss: 0.6215 - accuracy: 0.6601 - recall: 0.6559
Epoch 8/10
79/79 [==============================] - 0s 921us/step - loss: 0.6164 - accuracy: 0.6620 - recall: 0.6663
Epoch 9/10
79/79 [==============================] - 0s 933us/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[727, 432],
       [225, 316]], dtype=int64)

In [41]:
# ROS 
model10 = create_mlp(64, 'adam', X_train_scaled_encoded)
model10.fit(X_resampled_ros, y_resampled_ros, epochs=10, batch_size=32)

y_pred = model10.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("ROS 1 hidden, 64 nodes, adam", y_test, y_pred)
cm

Epoch 1/10
169/169 [==============================] - 1s 849us/step - loss: 0.6713 - accuracy: 0.5882 - recall: 0.5808
Epoch 2/10
169/169 [==============================] - 0s 873us/step - loss: 0.6428 - accuracy: 0.6324 - recall: 0.6026
Epoch 3/10
169/169 [==============================] - 0s 879us/step - loss: 0.6347 - accuracy: 0.6443 - recall: 0.6371
Epoch 4/10
169/169 [==============================] - 0s 885us/step - loss: 0.6289 - accuracy: 0.6463 - recall: 0.6293
Epoch 5/10
169/169 [==============================] - 0s 873us/step - loss: 0.6205 - accuracy: 0.6567 - recall: 0.6435
Epoch 6/10
169/169 [==============================] - 0s 896us/step - loss: 0.6172 - accuracy: 0.6576 - recall: 0.6625
Epoch 7/10
169/169 [==============================] - 0s 926us/step - loss: 0.6133 - accuracy: 0.6624 - recall: 0.6593
Epoch 8/10
169/169 [==============================] - 0s 879us/step - loss: 0.6102 - accuracy: 0.6644 - recall: 0.6454
Epoch 9/10
169/169 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[798, 361],
       [261, 280]], dtype=int64)

In [42]:
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
8,"RUS 1 hidden, 64 nodes, adam",0.584,0.490,0.422,0.614
9,"ROS 1 hidden, 64 nodes, adam",0.518,0.474,0.437,0.634
4,"1 hidden, 100 nodes, adam",0.307,0.380,0.498,0.681
2,"1 hidden, 64 nodes per, adam, scaled",0.283,0.358,0.487,0.677
6,"1 hidden, 250 nodes, adam",0.274,0.343,0.458,0.666
7,"1 hidden, 300 nodes, adam",0.272,0.339,0.451,0.663
0,"1 hidden, 64 nodes per, adam",0.246,0.329,0.498,0.681
5,"1 hidden, 32 nodes, adam",0.194,0.279,0.495,0.681
1,"1 hidden, 64 nodes per, adagrad",0.179,0.268,0.533,0.689
3,"1 hidden, 64 nodes per, adagrad, scaled",0.176,0.260,0.497,0.681


In [43]:
# RUS 
model11 = create_mlp(64, 'adam', X_train_scaled_encoded)
model11.fit(X_resampled_rus, y_resampled_rus, epochs=10, batch_size=25)

y_pred = model11.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("RUS 1 hidden, 64 nodes, adam, batch 25", y_test, y_pred)
cm

Epoch 1/10
101/101 [==============================] - 0s 967us/step - loss: 0.6761 - accuracy: 0.5796 - recall: 0.5713
Epoch 2/10
101/101 [==============================] - 0s 908us/step - loss: 0.6506 - accuracy: 0.6280 - recall: 0.5957
Epoch 3/10
101/101 [==============================] - 0s 888us/step - loss: 0.6446 - accuracy: 0.6307 - recall: 0.6235
Epoch 4/10
101/101 [==============================] - 0s 878us/step - loss: 0.6376 - accuracy: 0.6335 - recall: 0.6178
Epoch 5/10
101/101 [==============================] - 0s 878us/step - loss: 0.6336 - accuracy: 0.6450 - recall: 0.6200
Epoch 6/10
101/101 [==============================] - 0s 878us/step - loss: 0.6275 - accuracy: 0.6470 - recall: 0.6466
Epoch 7/10
101/101 [==============================] - 0s 908us/step - loss: 0.6240 - accuracy: 0.6561 - recall: 0.6452
Epoch 8/10
101/101 [==============================] - 0s 918us/step - loss: 0.6206 - accuracy: 0.6577 - recall: 0.6610
Epoch 9/10
101/101 [============================

C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[759, 400],
       [251, 290]], dtype=int64)

In [44]:
master_scores.sort_values('Recall', ascending=False).head()

,Model,Recall,F1,Precision,Accuracy
8,"RUS 1 hidden, 64 nodes, adam",0.584,0.490,0.422,0.614
10,"RUS 1 hidden, 64 nodes, adam, batch 25",0.536,0.471,0.420,0.617
9,"ROS 1 hidden, 64 nodes, adam",0.518,0.474,0.437,0.634
4,"1 hidden, 100 nodes, adam",0.307,0.380,0.498,0.681
2,"1 hidden, 64 nodes per, adam, scaled",0.283,0.358,0.487,0.677


Well we've gotten it pretty close to the performance of the LR model. Is anything about it better? Run speed?

In [45]:
import time
# training time 
start = time.time()
model10.fit(X_resampled_ros, y_resampled_ros, epochs=10, batch_size=32)
end = time.time()
training_time = end - start
training_time

Epoch 1/10
169/169 [==============================] - 0s 920us/step - loss: 0.5979 - accuracy: 0.6744 - recall: 0.6795
Epoch 2/10
169/169 [==============================] - 0s 885us/step - loss: 0.5936 - accuracy: 0.6807 - recall: 0.6889
Epoch 3/10
169/169 [==============================] - 0s 885us/step - loss: 0.5911 - accuracy: 0.6848 - recall: 0.7003
Epoch 4/10
169/169 [==============================] - 0s 890us/step - loss: 0.5890 - accuracy: 0.6831 - recall: 0.6854
Epoch 5/10
169/169 [==============================] - 0s 896us/step - loss: 0.5817 - accuracy: 0.6896 - recall: 0.6923
Epoch 6/10
169/169 [==============================] - 0s 885us/step - loss: 0.5802 - accuracy: 0.6890 - recall: 0.7093
Epoch 7/10
169/169 [==============================] - 0s 926us/step - loss: 0.5779 - accuracy: 0.6963 - recall: 0.7039
Epoch 8/10
169/169 [==============================] - 0s 890us/step - loss: 0.5757 - accuracy: 0.6989 - recall: 0.6993
Epoch 9/10
169/169 [============================

1.5937635898590088

# Comparing to a Logistic Regression
In Cat-pstone 1 I had did not prune down the columns nearly as much. I want to see how it does with this new training set. 

In [46]:
from sklearn.linear_model import LogisticRegression

log_balanced = LogisticRegression(class_weight='balanced').fit(X_train_encoded, y_train)
y_pred = log_balanced.predict(X_test_encoded)

# add_to_master(model_description, y_true, y_predicted, binary=True)
add_to_master("class_weight='balanced', Logistic Regression", y_test, y_pred)

log_balanced_scaled = LogisticRegression(class_weight='balanced').fit(X_train_scaled_encoded, y_train)
y_pred = log_balanced_scaled.predict(X_test_scaled_encoded)

# add_to_master(model_description, y_true, y_predicted, binary=True)
add_to_master("class_weight='balanced', Logistic Regression, scaled", y_test, y_pred);

C:\Users\gwent\anaconda3\envs\wool_sucking_nn\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)
C:\Users\gwent\anaconda3\envs\wool_sucking_nn\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

In [47]:
# for reference, cat-pstone 1 model was 0.622, 0.519, 0.446, 0.635
master_scores.sort_values('Recall', ascending=False).head()

,Model,Recall,F1,Precision,Accuracy
12,"class_weight='balanced', Logistic Regression, ...",0.593,0.504,0.437,0.628
11,"class_weight='balanced', Logistic Regression",0.588,0.502,0.438,0.629
8,"RUS 1 hidden, 64 nodes, adam",0.584,0.490,0.422,0.614
10,"RUS 1 hidden, 64 nodes, adam, batch 25",0.536,0.471,0.420,0.617
9,"ROS 1 hidden, 64 nodes, adam",0.518,0.474,0.437,0.634


Given that this model does not perform as well as the cat-pstone 1 model (marginally), I want to see how my best performing model architecture does on the dataset I used with cat-pstone 1 (with a lot more columns!). 

In [48]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('..\Data\cat_clean.csv')

In [49]:
# binarize wool-sucking
df_bin = df.copy()
df_bin['ws_binary'] = df_bin['Wool_sucking'].replace([1, 2, 3, 4, 5, 6, 7], [1, 1, 1, 1, 1, 1, 1])
df_bin.drop(columns='Wool_sucking', inplace=True)

# Recode Behaviour_problem to binary
df_bin['Behaviour_problem'].replace([1, 2, 3], [0, 1, 1], inplace=True)

# encode breed group
df_bin_encoded = pd.get_dummies(df_bin, columns=['Breed_group'])

In [50]:
X = df_bin_encoded.drop(columns=['ws_binary'])
y = df_bin_encoded['ws_binary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

# making agg columns for training set 
X_train['agg_shy'] = (X_train['Shyness_novel'] + X_train['Shyness_strangers']) / 2
X_train['agg_aggress_people'] = (X_train['Aggression_owner'] + X_train['Aggression_stranger']) / 2 
X_train.drop(columns=['Shyness_novel', 'Shyness_strangers', 'Aggression_stranger', 'Aggression_owner'], inplace=True)
X_train.columns

# making agg columns for testing set
X_test['agg_shy'] = (X_test['Shyness_novel'] + X_test['Shyness_strangers']) / 2
X_test['agg_aggress_people'] = (X_test['Aggression_owner'] + X_test['Aggression_stranger']) / 2
X_test.drop(columns=['Shyness_novel', 'Shyness_strangers', 'Aggression_stranger', 'Aggression_owner'], inplace=True)
X_test.columns

Index(['Age', 'Gender', 'Neuter_status', 'Weaning_age', 'Outdoors',
       'Other_cats', 'Activity_level', 'Contact_people', 'Aggression_cats',
       'Grooming', 'Behaviour_problem', 'Breed_group_ABY', 'Breed_group_BEN',
       'Breed_group_BRI', 'Breed_group_BUR', 'Breed_group_CRX',
       'Breed_group_DRX', 'Breed_group_EUR', 'Breed_group_HCS',
       'Breed_group_KOR', 'Breed_group_MCO', 'Breed_group_NFO',
       'Breed_group_ORI', 'Breed_group_PER', 'Breed_group_RAG',
       'Breed_group_RUS', 'Breed_group_SBI', 'Breed_group_SIB',
       'Breed_group_TUV', 'Breed_group_other', 'agg_shy',
       'agg_aggress_people'],
      dtype='object')

In [51]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled_encoded = scaler.transform(X_train)
X_test_scaled_encoded = scaler.transform(X_test)

rus = RandomUnderSampler()
X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train_scaled_encoded, y_train)

model12 = create_mlp(64, 'adam', X_resampled_rus)
model12.fit(X_resampled_rus, y_resampled_rus, epochs=10, batch_size=32)

y_pred = model12.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
add_to_master("old data rus adam 64", y_test, y_pred)
cm

Epoch 1/10
79/79 [==============================] - 0s 869us/step - loss: 0.6732 - accuracy: 0.5879 - recall: 0.5995
Epoch 2/10
79/79 [==============================] - 0s 908us/step - loss: 0.6317 - accuracy: 0.6378 - recall: 0.6481
Epoch 3/10
79/79 [==============================] - 0s 869us/step - loss: 0.6178 - accuracy: 0.6516 - recall: 0.6173
Epoch 4/10
79/79 [==============================] - 0s 869us/step - loss: 0.6058 - accuracy: 0.6631 - recall: 0.6513
Epoch 5/10
79/79 [==============================] - 0s 869us/step - loss: 0.5969 - accuracy: 0.6857 - recall: 0.6854
Epoch 6/10
79/79 [==============================] - 0s 882us/step - loss: 0.5890 - accuracy: 0.6789 - recall: 0.6733
Epoch 7/10
79/79 [==============================] - 0s 869us/step - loss: 0.5809 - accuracy: 0.6956 - recall: 0.6974
Epoch 8/10
79/79 [==============================] - 0s 882us/step - loss: 0.5713 - accuracy: 0.7019 - recall: 0.6942
Epoch 9/10
79/79 [==============================] - 0s 895us/ste

C:\Users\gwent\AppData\Local\Temp\ipykernel_4332\1005917253.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_scores = master_scores.append(new_row, ignore_index=True)


array([[619, 548],
       [176, 366]], dtype=int64)

In [52]:
# for reference, cat-pstone 1 model was 0.622, 0.519, 0.446, 0.635
# it really doesn't do any better than a weighted LR model, in either dataset
master_scores.sort_values('Recall', ascending=False)

,Model,Recall,F1,Precision,Accuracy
13,old data rus adam 64,0.675,0.503,0.400,0.576
12,"class_weight='balanced', Logistic Regression, ...",0.593,0.504,0.437,0.628
11,"class_weight='balanced', Logistic Regression",0.588,0.502,0.438,0.629
8,"RUS 1 hidden, 64 nodes, adam",0.584,0.490,0.422,0.614
10,"RUS 1 hidden, 64 nodes, adam, batch 25",0.536,0.471,0.420,0.617
9,"ROS 1 hidden, 64 nodes, adam",0.518,0.474,0.437,0.634
4,"1 hidden, 100 nodes, adam",0.307,0.380,0.498,0.681
2,"1 hidden, 64 nodes per, adam, scaled",0.283,0.358,0.487,0.677
6,"1 hidden, 250 nodes, adam",0.274,0.343,0.458,0.666
7,"1 hidden, 300 nodes, adam",0.272,0.339,0.451,0.663
